In [ ]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


In [ ]:
import math
import torch
import pickle
import numpy as np
import pandas as pd
#import seaborn as sns
import matplotlib.pyplot as plt
from sklearn import preprocessing
from datetime import datetime, timedelta
from sklearn.preprocessing import MinMaxScaler
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import TimeSeriesSplit
from sklearn.metrics import mean_squared_error, mean_absolute_percentage_error, mean_absolute_error

import joblib

# check xgboost version
from xgboost import XGBRegressor

#torch stuff
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader

#from torch_geometric.utils import dense_to_sparse


import lightgbm as lgb
from lightgbm import LGBMRegressor
import holidays
us_holidays = holidays.country_holidays('US')

pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)

directory = '/content/drive/MyDrive/Thesis'
data_dir = directory + "/Data"
models_dir = directory + "/models"

/usr/local/lib/python3.10/dist-packages/dask/dataframe/__init__.py:42: FutureWarning: 
Dask dataframe query planning is disabled because dask-expr is not installed.

You can install it with `pip install dask[dataframe]` or `conda install dask`.
This will raise in a future version.

  warnings.warn(msg, FutureWarning)


In [ ]:
from hyperopt.pyll.base import scope
from hyperopt import fmin, tpe, hp, STATUS_OK, Trials

In [ ]:
suf = "_normal_xgb_11_08"

returns = pd.read_csv(f"{models_dir}/test_returns_predictions{suf}.csv").rename(
    columns={
        "end_station_cluster": "start_station_cluster",
        "pred": "returns"
        }
    )[['started_at_hourly', 'returns', 'start_station_cluster']]

In [ ]:
df = pd.read_csv(f'{data_dir}/final_model_input_partial_scale_4.csv')
df = df.merge(returns, on=['started_at_hourly', 'start_station_cluster'], how='left').fillna(0)

df['started_at_hourly'] = pd.to_datetime(df['started_at_hourly'])
df = df.sort_values(by=['start_station_cluster', 'started_at_hourly'])

df_test = df[pd.to_datetime(
    df['started_at_hourly']) >= pd.to_datetime("2024-01-01 00:00:00")]
df_train = df[pd.to_datetime(
    df['started_at_hourly']) < pd.to_datetime("2024-01-01 00:00:00")]

del df

In [ ]:
target_scaler = joblib.load(f'{models_dir}/target_scaler.sav')

In [ ]:
execfile('/content/drive/MyDrive/Thesis/models_training/model_training.py')

Using device: cpu


# GNN Variance Embedding

In [ ]:
FEATURES = ['start_station_cluster', 'started_at_year',
            'started_at_month', 'started_at_day',
            'started_at_hour',
            'started_at_week',
            'started_at_quarter',
            'started_at_dayofweek',
            'is_holiday',
            #'flag_added',
            "temp",
            "dwpt",
            "rhum",
            "prcp",
            "wdir",
            "wspd",
            "pres",
            "coco",
            "demand_lag_1_h",
            "demand_lag_2_h",
            "demand_lag_24_h",
            "temp_lag_1_h",
            "temp_lag_2_h",
            "temp_lag_24_h",
            "prcp_lag_1_h",
            "prcp_lag_2_h",
            "prcp_lag_24_h",
            "rhum_lag_1_h",
            "rhum_lag_2_h",
            "rhum_lag_24_h",
            "wspd_lag_1_h",
            "wspd_lag_2_h",
            "wspd_lag_24_h",
            "mean_gnn_cluster_demand_1h",
            "total_gnn_cluster_demand_1h",
            "total_demand_1h",
            "demand_degrees_1h",
            "returns"
]
FEATURES.extend([f"dim_mean_{i}" for i in range(50)])

## XGBoost

In [ ]:
model, model_preds = train_model(df_train, df_test, FEATURES, model='xgb')

overall
MSE: 3.5229701168797694
RMSE: 1.876957675836024
MAE: 0.9956896304822397
MAPE: 0.39470800631747033

Non-zero
MSE: 7.185092966319084
RMSE: 2.6805023720040024
MAE: 1.6903708260908172
MAPE: 0.35436177838032124

Zeros
MSE: 0.5287028726317953
RMSE: 0.7271195724444469
MAE: 0.42769635589172467
MAPE: 0.42769635589172467


In [ ]:
#11-09 scaled
3.511820258766204
1.873985127680101
0.3926758989622167

#11-28
3.516988135177505
1.875363467485038
0.3956547845817011

0.3956547845817011

In [ ]:
suf = "_m_xgb_01_11"


model_preds.to_csv(f"{models_dir}/test_predictions{suf}.csv")
del model_preds

filename = f'{models_dir}/demand_model{suf}.sav'
joblib.dump(model, filename)
del model

## LightGBM

In [ ]:
model, model_preds = train_model(df_train, df_test, FEATURES, model='lgbm')

[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 1.229155 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 16454
[LightGBM] [Info] Number of data points in the train set: 1140300, number of used features: 87
[LightGBM] [Info] Start training from score 1.005143
overall
MSE: 3.6140499072235297
RMSE: 1.9010654663171203
MAE: 1.0059431335721514
MAPE: 0.3970047663618654

Non-zero
MSE: 7.387745759151811
RMSE: 2.7180407942398164
MAE: 1.70922724282392
MAPE: 0.355529999215768

Zeros
MSE: 0.5285570541060253
RMSE: 0.7270192941772765
MAE: 0.4309158452180498
MAPE: 0.4309158452180498


In [ ]:
#11-28
3.6151918646219734
1.9013657892741138
0.3974178856605776

0.3974178856605776

In [ ]:
suf = "_m_lgbm_01_11"


model_preds.to_csv(f"{models_dir}/test_predictions{suf}.csv")
del model_preds

filename = f'{models_dir}/demand_model{suf}.sav'
joblib.dump(model, filename)
del model

## Random Forest

In [ ]:
model, model_preds = train_model(df_train, df_test, FEATURES, model='rf')

In [ ]:
#11_01
3.6042873699256828
1.8984960810930538
0.4084309933647605

#11_28
3.7861083417174686
1.9457924713898624
0.437449284109053

In [ ]:
suf = "_m_rf_01_11"


model_preds.to_csv(f"{models_dir}/test_predictions{suf}.csv")
del model_preds

filename = f'{models_dir}/demand_model{suf}.sav'
joblib.dump(model, filename)
del model